## DMCX 经济模型模拟包含的内容

- DMCX的释放模型，GWT兑换率变化
- DMC DAO的分红逻辑
- 公共数据挖矿的模型，包括 Sponosr的充值计算，矿工的质押计算，矿工的收益计算
- 公共数据挖矿的奖池分配模型（总金额，指定名次的奖励计算）
- 私有数据挖矿的模型，主要关注点也是矿工的收益计算和质押计算

## DMCX 的释放模型

从2024年8月开始一共有420个周期，每个周期大概是现实世界的1周。每21个周期进行一次难度调整。难度调整后的周期释放的Token是上一个周期的80%。总释放为5亿DMC。

第1-21周，每周最多释放480万枚DMC。随后的第21-42周，每周最多释放480万*0.8枚DMC

In [1]:
global exchange_cycle_data 
exchange_cycle_data = []
global unreleased_cycle_count 
unreleased_cycle_count = 0

def push_cycle_data(cycle_total):
    for i in range(21):
        cycle_data = {}
        cycle_data["exchange_gwt"] = 0
        cycle_data["ex_rate"] = 210
        cycle_data["dmcx_released"] = 0
        cycle_data['dmcx_balance'] = cycle_total
        exchange_cycle_data.append(cycle_data)

def print_dmx_release_table():
    n = 1
    count = 1
    for i in range(19):
        n = n * 0.8
        count = count + n
        

    init = ((50000*10000) / count) / 21
    print("first 1-21 cycle ,every cycle(week) relase dmcx:",init)
    push_cycle_data(init)

    token_count = init * 21
    balance = token_count
    cycle_index = 22
    for i in range (1,20):
        balance = balance * 0.8
        print(f"cycle {cycle_index} - {cycle_index+20} ,every cycle(week) release dmcx:",balance)
        push_cycle_data(balance)
        token_count += balance
        cycle_index += 21

    print("total release DMCX:",token_count)
    print("total not released all DMCX cycle count:",unreleased_cycle_count)

print_dmx_release_table()

first 1-21 cycle ,every cycle(week) relase dmcx: 4817446.134360656
cycle 22 - 42 ,every cycle(week) release dmcx: 80933095.05725902
cycle 43 - 63 ,every cycle(week) release dmcx: 64746476.04580722
cycle 64 - 84 ,every cycle(week) release dmcx: 51797180.83664578
cycle 85 - 105 ,every cycle(week) release dmcx: 41437744.66931663
cycle 106 - 126 ,every cycle(week) release dmcx: 33150195.735453304
cycle 127 - 147 ,every cycle(week) release dmcx: 26520156.588362645
cycle 148 - 168 ,every cycle(week) release dmcx: 21216125.270690117
cycle 169 - 189 ,every cycle(week) release dmcx: 16972900.216552094
cycle 190 - 210 ,every cycle(week) release dmcx: 13578320.173241675
cycle 211 - 231 ,every cycle(week) release dmcx: 10862656.13859334
cycle 232 - 252 ,every cycle(week) release dmcx: 8690124.910874672
cycle 253 - 273 ,every cycle(week) release dmcx: 6952099.928699738
cycle 274 - 294 ,every cycle(week) release dmcx: 5561679.942959791
cycle 295 - 315 ,every cycle(week) release dmcx: 4449343.9543678


### getCircleBalance@exchange.sol

```solidity
initial_dmc_balance = 4817446 ether;

function getCircleBalance(uint256 circle) public view returns (uint256) {
    //return 210 ether;

    uint256 adjust_times = (circle-1) / adjust_period;
    uint256 balance = initial_dmc_balance;
    for (uint i = 0; i < adjust_times; i++) {
        balance = balance * 4 / 5;
    }
    return balance;
}
```

## DMCX<->GWT的兑换

通过exchange.sol的接口，用户可以在一个周期里进行GWT<->DMCX的兑换。GWT和DMCX的兑换比率(ex_rate)是动态变化的.ex_rate每个周期开始时确定，在整个周期内不变。

基于该ex_rate:

- 可以将1000个GWT兑换成 ( 1000 / ex_rate) 个DMCX，兑换得到的DMCX来自本cycle待释放的DMCX（如果待释放DMCX消耗完了该调用会失败）,消耗的GWT会进入DMC DAO的分红池。
- 可以将1000个DMCX兑换成 ( 1000 * ex_rate * 1.1) 个GWT，该调用总是会成功,消耗的DMCX会被销毁。

### ex_rate的变化
ex_rate变化的基本逻辑是：
- 如果一个cycle里的DMCX都被兑换完了，那么下一个cycle的ex_rate就会增加,增加的比率和兑换完的时间有关，兑换的速度越快则ex_rate增加的越多，最多增加20%。
- 如果一个cycle里的DMCX没有兑换完，那么下一个cycle的ex_rate就会减少，剩余的未兑换DMCX越多则ex_rate下降的越多。ex_rate的下降最多为-20%。未兑换完成的DMCX会平滑的分配到未来的一些周期里，影响这些周期的可释放DMCX数量。



In [2]:

# if dmcx_released = 0,说明该周期的dmcx都释放了,used_time的值为0-7，说明是第几天挖完的

def set_dmcx_released(cycle_index, dmcx_released,used_time):
    global unreleased_cycle_count
    global exchange_cycle_data

    cycle_data= exchange_cycle_data[cycle_index - 1]
    next_cycle_data = exchange_cycle_data[cycle_index]

    
    if dmcx_released == 0:
        cycle_data["dmcx_released"] = cycle_data["dmcx_balance"]
        cycle_data["used_time"] = used_time
        new_rate = (7 - used_time) / 7
        if new_rate > 0.2:
            new_rate = 0.2
        next_cycle_data["ex_rate"] = cycle_data["ex_rate"] * (1+new_rate)
    else:
        cycle_data["dmcx_released"] = dmcx_released
        new_rate = dmcx_released / cycle_data["dmcx_balance"]
        if new_rate < 0.8:
            new_rate = 0.8
        next_cycle_data["ex_rate"] = cycle_data["ex_rate"] * new_rate
        addtion_dmcx = cycle_data["dmcx_balance"] - dmcx_released
        unreleased_cycle_count = unreleased_cycle_count + 1
        for i in range (unreleased_cycle_count):
            exchange_cycle_data[cycle_index + i]["dmcx_balance"] += (addtion_dmcx / unreleased_cycle_count)

    cycle_data["exchange_gwt"] = cycle_data["ex_rate"] * cycle_data["dmcx_released"]


def show_cycle_data(max_cycle):
    for i in range(max_cycle):
        cycle_data = exchange_cycle_data[i]
        if cycle_data["dmcx_released"] == cycle_data["dmcx_balance"]:
            print(f"all released cycle {i+1} dmcx_released:{cycle_data['dmcx_released']} exchange_gwt:{cycle_data['exchange_gwt']} ex_rate:{cycle_data['ex_rate']} dmcx_balance:{cycle_data['dmcx_balance']} used_time:{cycle_data['used_time']}")
        else:
            print(f"cycle {i+1} dmcx_released:{cycle_data['dmcx_released']} exchange_gwt:{cycle_data['exchange_gwt']} ex_rate:{cycle_data['ex_rate']} dmcx_balance:{cycle_data['dmcx_balance']}")
    
    return 


set_dmcx_released(1,0,6)
set_dmcx_released(2,0,5)
set_dmcx_released(3,0,3)
set_dmcx_released(4,0,3.5)
set_dmcx_released(5,0,6.3)
set_dmcx_released(6,4800000,0)
set_dmcx_released(7,2800000,0)
set_dmcx_released(8,2800000,0)
set_dmcx_released(9,0,6.5)
set_dmcx_released(10,0,6.9)

show_cycle_data(10)




all released cycle 1 dmcx_released:4817446.134360656 exchange_gwt:1011663688.2157378 ex_rate:210 dmcx_balance:4817446.134360656 used_time:6
all released cycle 2 dmcx_released:4817446.134360656 exchange_gwt:1156187072.2465575 ex_rate:240.0 dmcx_balance:4817446.134360656 used_time:5
all released cycle 3 dmcx_released:4817446.134360656 exchange_gwt:1387424486.695869 ex_rate:288.0 dmcx_balance:4817446.134360656 used_time:3
all released cycle 4 dmcx_released:4817446.134360656 exchange_gwt:1664909384.0350425 ex_rate:345.59999999999997 dmcx_balance:4817446.134360656 used_time:3.5
all released cycle 5 dmcx_released:4817446.134360656 exchange_gwt:1997891260.8420513 ex_rate:414.71999999999997 dmcx_balance:4817446.134360656 used_time:6.3
cycle 6 dmcx_released:4800000 exchange_gwt:2189721600.0 ex_rate:456.192 dmcx_balance:4817446.134360656
cycle 7 dmcx_released:2800000 exchange_gwt:1272711787.3241568 ex_rate:454.53992404434166 dmcx_balance:4834892.268721312
cycle 8 dmcx_released:2800000 exchange_g

### adjustExchangeRate@exchange.sol

```solidity
function adjustExchangeRate() internal {
    if(block.timestamp >= current_mine_circle_start + min_circle_time) {
        //end current cycle, calculate new exchange rate
        uint256 old_rate = dmc2gwt_rate;
        if(remain_dmc_balance > 0) {
            total_addtion_dmc_balance += remain_dmc_balance;
            addtion_circle_count += 1;

            // console.log("prev cycle dmc balance left %d, total left %d, total left cycle %d", remain_dmc_balance, total_addtion_dmc_balance, addtion_circle_count);

            //there has remaining DMCs in current cycle, decrease DMC -> GWT exchange rate
            dmc2gwt_rate = dmc2gwt_rate * (1-remain_dmc_balance/current_circle_dmc_balance);
            if (dmc2gwt_rate < old_rate * 4 / 5) {
                // we have a limit down as 20%
                dmc2gwt_rate = old_rate * 4 / 5;
            }
            if(dmc2gwt_rate < 210) {
                // the lowest rate is 210
                dmc2gwt_rate = 210;
            }
            // console.log("decrease dmc2gwt_rate to %d", dmc2gwt_rate);
        } else {
            if (addtion_circle_count > 0) {
                addtion_circle_count -= 1;
            }
            // all DMCs in current cycle have been mined, increase DMC -> GWT exchange rate
            dmc2gwt_rate = dmc2gwt_rate * (1+(current_finish_time-current_mine_circle_start)/min_circle_time);
            if(dmc2gwt_rate > old_rate * 6 / 5) {
                // we have a raising limit as 20%
                dmc2gwt_rate = old_rate * 6 / 5;
            }
            // for test
            // dmc2gwt_rate = 210;
            // console.log("increase dmc2gwt_rate to %d", dmc2gwt_rate);
        }

        emit gwtRateChanged(dmc2gwt_rate, old_rate);

        _newCycle();
    } else {
        // console.log("keep cycle.");
        require(remain_dmc_balance > 0, "no dmc balance in current circle");
    }
}
```

## 质押DMCX的分红奖励

DMCX的持有者可以将自己的DMCX质押到DMC DAO中，质押的DMCX会参与分红。分红的奖励来自于DMC DAO的分红池.其基本规则是：
- dividend_cycle: 分红周期，每次DMC DAO的分红池获得收入都会打入当前周期。获得收入时，还会检查当前时间与本周期启动时间的差，如果差大于3天，则开始一个新的分红周期，新的divident_cycle是当前cycle,并把上一个cycle标记为结束。
- 已结束的cycle里有确定的总收入和总质押DMCX数。
- 用户质押的DMCX，总是进入下一个分红周期（要等待下一个分红周期的开始）
- 用户可以从已结束的cycle里提取自己的分红奖励，提取的奖励是 (用户在该cycle质押的DMCX数 / 该cycle总质押DMCX数) * 该cycle总收入


In [3]:

global dividend_cycle_data 
dividend_cycle_data = []

def add_divident_cycle(total_stack,income):
    global dividend_cycle_data
    dividend_cycle_data.append({"dmcx":total_stack,"income":income})

def test_dividend(pelegement_cycle_index,end_cycle_index,dmcx_count):
    global dividend_cycle_data
    my_income = 0
    for i in range(pelegement_cycle_index+1,end_cycle_index+1):
        this_cycle_data = dividend_cycle_data[i]
        this_income = this_cycle_data["income"] * (dmcx_count / this_cycle_data["dmcx"])
        my_income += this_income
        print(f"pelgement {dmcx_count} dmcx at cycle:{i},income:{this_income}")

    print(f"pelgement {dmcx_count} dmcx from cycle:{pelegement_cycle_index} - {end_cycle_index} ,income:{my_income}")
    
add_divident_cycle(0,0)
add_divident_cycle(10000,1000)
add_divident_cycle(15000,2000)
add_divident_cycle(20000,3000)
add_divident_cycle(30000,4000)

test_dividend(1,3,5000)




pelgement 5000 dmcx at cycle:2,income:666.6666666666666
pelgement 5000 dmcx at cycle:3,income:750.0
pelgement 5000 dmcx from cycle:1 - 3 ,income:1416.6666666666665


### tryNewCycle@dividend.sol
    
```solidity
cycleMinLength = 3 days;
/**
    * Check if the new cycle should be started on check point
    * If the current cycle is over the max length, then start a new cycle
*/
function tryNewCycle() public {
    uint256 currentBlocktime = block.timestamp;
    
    CycleInfo storage currentCycle = cycles[currentCycleIndex];
    if (currentBlocktime - currentCycle.startBlocktime >= cycleMinLength) {
        currentCycleIndex = currentCycleIndex + 1;
        console.log("enter new cycle %d, totalStaked %d", currentCycleIndex, totalStaked);
        CycleInfo storage newCycle = cycles[currentCycleIndex];
        newCycle.startBlocktime = currentBlocktime;
        newCycle.totalStaked = totalStaked;
        
        if (currentCycle.totalStaked == 0) {
            newCycle.rewards = currentCycle.rewards;
        }

        emit NewCycle(currentCycleIndex, currentBlocktime);
    }
}
```

## 公共数据挖矿的经济模型

公共数据挖矿合约里一共有3类用户，分别是：Supplier，Data Owner，Data Supplier。Supplier保存了完整的公共数据，通过SHOW数据的存储证明来获得激励（公共数据挖矿），Data Owner是公共数据关联的NFT的Owner，Sponoser是公共数据的赞助商,通常是第一个将NFT加入到公共数据合约中的用户。

公共数据挖矿的基本流程是：

1. Sponsor创建公共数据,并给公共数据充入第一笔GWT余额。这笔GWT一部分会进入公共数据的本轮竞赛奖池。
2. Supplier质押GWT成为Supplier
3. Supplier在矿机上保存公共数据，并在适当的时机展示公共数据的存储证明(SHOW)，获得奖励。奖励一方面来自公共数据的余额，一方面来自于算力奖励。
4. Supplier在SHOW成功后，公共数据在本轮竞赛中的积分会增加。积分增加的多少取决于数据的大小。
5. 一段时间后本轮竞赛结束，分数最高的前32个公共数据会按比例瓜分本轮竞赛奖池。
6. 获胜数据的Sponsor,Data Owner ,Supplier可以按比例瓜分该数据的获奖金额

### 创建公共数据需要准备的GWT

创建公共数据的用户需要为公共数据充入第一笔GWT余额，为了保障后续的公共数据挖矿的正常运行，该余额有一定的最小值要求。最小值的计算公式如下：
```
创建时需要准备的GWT = 公共数据最小质押率(16) * 公共数据大小（单位GB） * 最小时长(96周) * 创建倍率(3倍)
```
首次充值的80%会成为公共数据的余额，20%会进入本轮竞赛奖池。


In [4]:
global public_datas
public_datas = {}

def get_min_publicdata_create_deposit_amount(public_data_size):
    pledgeRate = 16
    minWeeks = 96
    createDepositeRatio = 3
    if(public_data_size < 0.125):
        # min size is 0.125 GB
        public_data_size = 0.125

    return public_data_size * pledgeRate * minWeeks * createDepositeRatio 

# publid data size 的单位是GB
def create_public_data(data_hash,public_data_size):
    deposit_amount = get_min_publicdata_create_deposit_amount(public_data_size)
    data_balance = deposit_amount * 0.8
    system_reward = deposit_amount - data_balance
    public_datas[data_hash] = {"size":public_data_size,"balance":data_balance,"last_show":0,"pelgement":16}
    print(f"create public data {data_hash} ,size:{public_data_size} GB,deposit_amount:{deposit_amount} GWT",f"data_balance:{data_balance} GWT,system_reward:{system_reward} GWT")

create_public_data("hash0",0.01)
create_public_data("hash1",0.02)
create_public_data("hash2",4.0)
create_public_data("hash3",5.2)


create public data hash0 ,size:0.01 GB,deposit_amount:576.0 GWT data_balance:460.8 GWT,system_reward:115.19999999999999 GWT
create public data hash1 ,size:0.02 GB,deposit_amount:576.0 GWT data_balance:460.8 GWT,system_reward:115.19999999999999 GWT
create public data hash2 ,size:4.0 GB,deposit_amount:18432.0 GWT data_balance:14745.6 GWT,system_reward:3686.3999999999996 GWT
create public data hash3 ,size:5.2 GB,deposit_amount:23961.600000000002 GWT data_balance:19169.280000000002 GWT,system_reward:4792.32 GWT


### createPublicData@public_data_storage.sol

```solidity

function _dataSizeToGWT(uint64 dataSize) internal view returns (uint256) {
    uint64 fixedDataSize = dataSize;
    if (fixedDataSize < sysConfig.minDataSize) {
        fixedDataSize = sysConfig.minDataSize;
    }
    return (uint256(fixedDataSize) * 10 ** 18) >> 30;
}

uint64 dataSize = PublicDataProof.lengthFromMixedHash(dataMixedHash);
uint256 minAmount = pledgeRate *
    _dataSizeToGWT(dataSize) *
    sysConfig.minPublicDataStorageWeeks *
    sysConfig.createDepositRatio;
require(depositAmount >= minAmount, "deposit amount is too small");

// ...

uint256 balance_add = (depositAmount * 8) / 10;
publicDataInfo.dataBalance += balance_add;
uint256 system_reward = depositAmount - balance_add;
_addCycleReward(system_reward);
```

### Supplier(矿工) SHOW一个公共数据需要准备的GWT质押币

Supplier（矿工）是真正保存公共数据的人，他们把公共数据保存到本地，并在合适的时机SHOW存储证明以得到奖励。DMCX的存储证明基于ERC7585构造，要出示一个存储证明，就必须基一个“基准块高度”开始计算，在有效块高度前(目前是3个区块，在XLayer上大概是10秒，包含上链时间)完成计算并提交到链上就是一次有效的存储证明。ERC7585的机制里，链上的合约并不能100%验证存储证明的正确性，而是基于博弈模型：如果有人发现了一个更好的存储证明，那就说明之前上链的是Fake的存储证明（挑战成功）。挑战成功的矿工会拿走提交Fake证明用户的质押币。

基于上述流程，矿工在开始公共数据挖矿之前必须需要准备一定的质押币。质押币的需求和其潜在收益有关，公式如下：


***Normal模式GAS费高，质押币需求低***

这里的质押是根据SHOW 数据大小的一般性质押，需要的质押币数量是：
```
pledgeRate *_dataSizeToGWT(dataSize) * sysConfig.minLockWeeks 
```
Normal模式下，Show成功时间的计算是第二次合约调用，因此对于有大量balance的公共数据使用Normal模式无法快人一步得到更大的收益。

***immediately模式 GAS费低，质押币需求高***

这里的质押是指根据SHOW收益决定的质押，和数据的Balance有关。需要锁定的Supplier的质押币数量是:
```
(datablance*0.1*2) + pledgeRate *_dataSizeToGWT(dataSize) * sysConfig.minPublicDataStorageWeeks 
```
当datablance过低时，立即模式附加的GWT需求至少为210 GWT。

### 停止SHOW（冷却）等待质押币解锁

每次SHOW公共数据后，Supplier的质押币都会按上述公私被冻结一部分，并更新冻结时间。因此如果Supplier有足够多的质押币的情况下，Supplier可以同时SHOW多个公共数据。

当当前时间大于冻结时间时，所有的质押币都会解冻并可用。因此Supplier的策略应该是一口气SHOW多个公共数据，把可用质押币都用完，同时得到了GWT奖励。这些奖励可以立刻质押进来获得更多的非冻结质押币，也可以一段时间内部SHOW，等待质押币解锁后再开始下一批的SHOW。


In [5]:

def get_min_stake_amount(data_hash):
    global public_datas
    
    minLockWeeks = 24
    public_data = public_datas[data_hash]
    if public_data == None:
        return
    pledgeRate = public_data["pelgement"]
    min_stake_amount = public_data["size"] * pledgeRate * minLockWeeks
    return min_stake_amount

def get_immediate_stake_amount(data_hash):
    global public_datas
    
    minLockWeeks = 24
    public_data = public_datas[data_hash]
    if public_data == None:
        return
    pledgeRate = public_data["pelgement"]
    addtion = public_data["balance"] * 0.2
    if addtion < 210:
        addtion = 210
    min_stake_amount = public_data["size"] * pledgeRate * minLockWeeks + addtion
    return min_stake_amount

def show_min_stack(data_hash):
    min_stake_amount = get_min_stake_amount(data_hash)
    immediate_stake_amount = get_immediate_stake_amount(data_hash)
    print(f"public data {data_hash} min stake amount:{min_stake_amount} GWT,immediate stake amount:{immediate_stake_amount} GWT")

show_min_stack("hash0")
show_min_stack("hash1")
show_min_stack("hash2")
show_min_stack("hash3")

public data hash0 min stake amount:3.84 GWT,immediate stake amount:213.84 GWT
public data hash1 min stake amount:7.68 GWT,immediate stake amount:217.68 GWT
public data hash2 min stake amount:1536.0 GWT,immediate stake amount:4485.120000000001 GWT
public data hash3 min stake amount:1996.8000000000002 GWT,immediate stake amount:5830.656000000001 GWT


### _getLockAmountByHash@public_data_storage.sol

```solidity
function _getLockAmountByHash(
    bytes32 dataMixedHash,
    ShowType showType
) internal view returns (uint256, bool) {
    uint64 dataSize = PublicDataProof.lengthFromMixedHash(dataMixedHash);
    uint256 normalLockAmount = _dataSizeToGWT(dataSize) *
        _publicDatas[dataMixedHash].pledgeRate *
        sysConfig.minLockWeeks;
    if(showType == ShowType.Immediately) {
        uint256 immediatelyLockAmount = (_publicDatas[dataMixedHash].dataBalance * 2) / 10;
        immediatelyLockAmount < sysConfig.minImmediatelyLockAmount ? sysConfig.minImmediatelyLockAmount : immediatelyLockAmount;
        return (normalLockAmount + immediatelyLockAmount, true);
    } else {
        return (normalLockAmount, false);
    }
}
```

### Supplier(矿工) SHOW成功后的奖励

- 第一次SHOW成功得到的来自于公共数据余额的奖励

```
SHOW奖励 = public_data.balance * 0.1
```

- 非第一次SHOW成功得到的公共数据余额+算力奖励，算力奖励还会增加公共数据的余额

GWT算力奖励是DMCX经济模型里的关键组成部分。其基本规则是：
```
算力 = 距离上一次SHOW成功的时间 * 公共数据大小 * 公共数据价格倍率(16) * 质押率(pledgeRate)
当前周期总算力 = 当前周期总算力 + 算力
算力奖励 = 算力 * 算力奖励系数 * 0.8
公共数据余额增加 = 算力 * 算力奖励系数 * 0.2
```

算力奖励系数是一个和两次SHOW时间有关的函数，其设计逻辑在于通过首次SHOW和当前SHOW所在周期的平均总算力和平均算力增长率，通过一个公式得到一个算力奖励系数。算力奖励系数的计算公式的基本原则是与平均总算力成反比，与平均算力增长率成正比，公式如下：

```python
def getGWTDifficultRatio(total_size,growth_rate):
    m =  1024*1024 - 1024*8*growth_rate
    result = (0.243*m) / (total_size+0.867*m) + 0.02

    return result
```



### SHOW与挑战

基于同一个区块，第一次提交公共数据的存储证明被称作SHOW，非第一次则为视为挑战。如果挑战展示的存储证明比上一次的存储证明更好，则挑战成功，挑战者能立刻拿走上一个存储证明提供者的质押币(Normal模式和immediately模式的总数不同)，并锁定按自己选择模式所需的质押币。否则挑战失败，挑战失败后挑战者只损失了手续费。在SHOW调用后，如果一段时间里没有人挑战成功，则SHOW最终成功，此时使用Normal模式SHOW的矿工可以发起withdrawShow调用获得SHOW奖励。



In [239]:
from datetime import datetime
import random

global compute_power_cycles
compute_power_cycles = []


def covert_timestr_to_timestamp(timestr):
    dt = datetime.strptime(timestr, "%Y-%m-%d %H:%M:%S")
    return dt.timestamp()


# from 0.02 to 0.3
def getGWTDifficultRatio(total_size,growth_rate):
    m =  1024*1024 - 1024*8*growth_rate
    result = (0.243*m) / (total_size+0.867*m) + 0.02
    print(f"total_size:{total_size},growth_rate:{growth_rate},difficult ratio:{result}")
    return result

# duration 单位是周
def cacl_compute_power(datasize,pledgement,duration,price=16):
    if datasize < 1:
        datasize = 1
    return datasize * pledgement * duration * price

def get_power_cycle_index_by_timestamp(timestamp):
    start_time = covert_timestr_to_timestamp("2024-07-24 00:00:00")
    # 一个算力周期是3天，和后面的排行榜周期一致
    cycle_index = int((timestamp - start_time) / (3*24*3600))
    return cycle_index

def add_compute_power_cycle(total_power):
    global compute_power_cycles
    cycle_data = {}
    cycle_data["total_power"] = total_power
    if len(compute_power_cycles) > 0:
        last_power = compute_power_cycles[-1]["total_power"]
        if total_power > last_power*1.005:
            cycle_data["d_power"] = (total_power - last_power) / last_power
            if cycle_data["d_power"] > 10:
                cycle_data["d_power"] = 10
        else:
            cycle_data["d_power"] = 0.005

    compute_power_cycles.append(cycle_data)

def showdata(hashdata,timestr):
    global public_datas
    data_info = public_datas[hashdata]
    if data_info == None:
        return
    timestamp = covert_timestr_to_timestamp(timestr)
    balance_reward = data_info["balance"] * 0.1
    public_datas[hashdata]["balance"] -= balance_reward
    cycle_index = get_power_cycle_index_by_timestamp(timestamp)
    last_show_time = public_datas[hashdata]["last_show"]

    is_power_show = False
    duration = (timestamp - last_show_time) / (7 * 24 * 3600)
    if last_show_time > 0:
        if duration > 1:
            is_power_show = True
    
    if is_power_show:
        last_show_cycle = get_power_cycle_index_by_timestamp(last_show_time)

        last_cycle_info = compute_power_cycles[last_show_cycle]
        this_cycle_info = compute_power_cycles[cycle_index-1]
        
        compute_power = cacl_compute_power(data_info["size"],data_info["pelgement"],duration)

        avg_power = (last_cycle_info["total_power"] + this_cycle_info["total_power"]) / 2
        avg_dpower = (last_cycle_info["d_power"] + this_cycle_info["d_power"]) / 2
        power_reward = compute_power * getGWTDifficultRatio(avg_power,avg_dpower) 

        compute_power_cycles[cycle_index]["total_power"] += compute_power
        public_datas[hashdata]["last_show"] = timestamp
        public_datas[hashdata]["balance"] += power_reward * 0.15
        #0.05的算力奖励会进入DMCX DAO的分红池
        
        print(f"show data {hashdata} get blance reward:{balance_reward},power:{compute_power},power reward:{power_reward*0.8}")
    else:
        public_datas[hashdata]["last_show"] = timestamp
        compute_power = cacl_compute_power(data_info["size"],data_info["pelgement"],1)
        compute_power_cycles[cycle_index]["total_power"] += compute_power

        print(f"show data {hashdata} get blance reward:",balance_reward)
        

add_compute_power_cycle(1)
add_compute_power_cycle(1)
totals = 1024*1024
for i in range(100):
    totals = totals + totals * random.uniform(-0.1,0.2)
    add_compute_power_cycle(totals)

show_min_stack("hash0")
show_min_stack("hash3")
showdata("hash0","2024-08-01 00:00:00")
showdata("hash3","2024-08-01 00:00:00")
showdata("hash0","2024-08-12 00:00:00")
showdata("hash3","2024-09-01 00:00:00")
showdata("hash3","2024-10-01 00:00:00")

public data hash0 min stake amount:3.84 GWT,immediate stake amount:213.84 GWT
public data hash3 min stake amount:1996.8000000000002 GWT,immediate stake amount:4821.72922698268 GWT
show data hash0 get blance reward: 38.087807115830266
show data hash3 get blance reward: 1412.46461349134
total_size:1225014.5142455557,growth_rate:5.036804229031415,difficult ratio:0.13665198936584874
show data hash0 get blance reward:34.27902640424724,power:402.2857142857143,power reward:43.97851452048344
total_size:1220534.497522133,growth_rate:5.0025,difficult ratio:0.13692058391200523
show data hash3 get blance reward:1271.2181521422062,power:5895.314285714287,power reward:645.751899475829
total_size:1551717.2603819617,growth_rate:0.042835620204552824,difficult ratio:0.12352194901444982
show data hash3 get blance reward:1156.2041850431574,power:5705.142857142857,power reward:563.7682920961221


## 公共数据排行榜

### 成功的SHOW会增加公共数据的积分

SHOW成功后数据在当前奖励周期的积分会增加，积分增加的数量只和数据的大小有关。1G以及一下的数据算1分，随后数据大小每次翻倍，积分增加1分。
挑战成功不会增加公共数据的积分。

```solidity
//TODO:这可能对大数据不够友好，需要修改
function _scoreFromHash(
    bytes32 dataMixedHash
) public pure returns (uint64) {
    uint256 size = PublicDataProof.lengthFromMixedHash(dataMixedHash) >> 30;
    if (size == 0) {
        // 1GB以下算1分
        return 1;
    }
    return uint64(size / 4 + 2);
}

//Increase according to the proportion of file size ， 0.1G - 1G 1，1G-4G 2， 4G-8G 3，8G-16G 4 16G-32G 5 ...
uint64 score = _scoreFromHash(dataMixedHash);
dataInfo.score += score;
```

### 排行榜规则

有了公共数据积分的规则，我们就可以按积分给公共数据排名。从原理上，排名越好的公共数据，其累计的有效存储证明数量也就越多，也就有更好的可靠性和可用性，是更优质的公共数据。DMCX通过公共数据排行榜奖励激励更多的公共数据变成优质的公共数据。

在XLayer上，排行榜的周期是固定区块数,约为3天。
```
sysConfig.blocksPerCycle = 86400;// Each cycle is 72 hours
```

每个排行榜周期开始时，都会重置所有公共数据在本周期的积分。排行榜还有一个进入排名的阈值（64分），未到达这个分值的公共数据无法进入排行榜。

排行榜的大小为32.

### 排行榜奖励

进入排行榜的公共数据可以瓜分本期排行榜的奖池。排行榜的奖池主要来源是给公共数据充值的20%GWT。以及上期排行榜的一部分。

```
公共数据本期奖励 = 本期奖池余额 * 0.8 * 本期百分比
```

第一名的本期百分比是15%，第二名是11.25%，第三名是9.3%... 第32名是0.8%
为了让每期的奖池更加稳定，本期奖池的15%会进入下期排行榜的奖池,5%进入DMC DAO的分红池。

#### 排行榜奖励的分配

当公共数据在一个奖励周期结束后进入了排行榜，任何人都可以发起合约调用来分配奖励（从利益相关来看，该操作一般是公共数据的Sponosr调用）。如果公共数据获得了100GWT的排行榜奖励，那么这100GWT的分配如下：

- 50%给Sponosr
- 20%给Data Owner, Data Owner的确认是根据PublicData关联的NFT的Owner来确认的
- 30%给最后Show的5个矿工，每个矿工可以拿到6%


### 抢获奖数据的Sponosr

任何用户都可以调用 addDeposit 给公共数据充值，当充值的金额超过公共数据最大单次充值的110%时，本次充值的用户会成为新的Sponosr。根据上述经济模型，在一个公共数据即将赢得排行榜时，如果潜在的奖励超过了成为公共数据Sponosr的成本，那么去抢Sponsor是一个有利可图的行为。

```solidity
function addDeposit(bytes32 dataMixedHash, uint256 depositAmount) {
    // ...
    if (depositAmount > ((publicDataInfo.maxDeposit * 11) / 10)) {
        publicDataInfo.maxDeposit = depositAmount;
        publicDataInfo.sponsor = msg.sender;
    }
}
```


In [ ]:
def get_winner_reward_percent(reword_rank):
    rewardScores = [
        240,
        180,
        150,
        120,
        100,
        80,
        60,
        53,
        42,
        36,
        35,
        34,
        33,
        32,
        31,
        30,
        29,
        28,
        27,
        26,
        25,
        24,
        23,
        22,
        21,
        20,
        19,
        18,
        17,
        16,
        15,
        14
    ]
    total_score = 0
    for idx in range(32):
        total_score += rewardScores[idx]

    score = rewardScores[reword_rank]
    return score / total_score

def win_reward(rank,reward_pool):
    reward_percent = get_winner_reward_percent(rank)
    total_reword =  reward_pool * reward_percent * 0.8
    print(f"rank {rank} ,pool reword:{reward_pool}\n\t total reward:{total_reword},sponsor reward:{total_reword * 0.5},owner reward:{total_reword * 0.2},supplier reward:{total_reword * 0.06}")

for i in range(32):
    print(f"reward rank {i} ,reward percent:{get_winner_reward_percent(i)}")
print("=============================================")
# 可以填入名次和奖池金额，查看预期的奖励分配
win_reward(12,140020)



##### withdrawReward@public_data_storage.sol

```solidity
    function withdrawReward(uint256 cycleNumber, bytes32 dataMixedHash) public {
        // Judging that the cycle of this time has ended
        //require(_currectCycle > cycleNumber, "cycle not finish");
        require(
            block.number > cycleNumber * sysConfig.blocksPerCycle + _startBlock,
            "cycle not finish"
        );
        CycleInfo storage cycleInfo = _cycleInfos[cycleNumber];
        CycleDataInfo storage dataInfo = cycleInfo.dataInfos[dataMixedHash];
        //REVIEW:The cost of GAS and 32 memory sorting in one time?
        uint256 scoreListRanking = cycleInfo.scoreList.getRanking(
            dataMixedHash
        );
        require(scoreListRanking > 0, "data not in rank");

        // No matter who take it, extract all the rewards at one time, and update the points
        require(dataInfo.score > 0, "already withdraw");

        // How many rewards to calculate
        uint256 totalReward = (cycleInfo.totalAward * 8) / 10;

        uint8 score = _getRewardScore(scoreListRanking);
        // If the total amount of data is less than 32, so excess rewards are precipitated in the contract account
        uint256 dataReward = (totalReward * score) / totalRewardScore;


        // transfoer 20% to owner
        gwtToken.transfer(
            _getDataOwner(dataMixedHash, _publicDatas[dataMixedHash]),
            dataReward / 5
        );

        // transfoer 50% to sponser
        gwtToken.transfer(_publicDatas[dataMixedHash].sponsor, dataReward / 2);

        // last showers
        uint256 showerReward = (dataReward - dataReward / 2 - dataReward / 5) /
            dataInfo.lastShowers.length;
        for (uint8 i = 0; i < dataInfo.lastShowers.length; i++) {
            gwtToken.transfer(dataInfo.lastShowers[i], showerReward);
        }

        // Setting Reward Have Been Taken
        dataInfo.score = 0;

        // Update points
        emit DataPointAdded(dataMixedHash, score);
        emit WithdrawReward(dataMixedHash, cycleNumber);
    }
}
```